In [1]:
import tensorflow as tf
import os
import scipy
from xml.etree.ElementTree import XML, fromstring
from pathlib import Path
import numpy as np

In [2]:
import tensorflow as tf
from pathlib import Path
import os
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import statistics

In [191]:
path_images = Path("/kaggle/input/face-mask-detection/images")
path_annot = Path("/kaggle/input/face-mask-detection/annotations")

In [192]:
image_paths = [
    str(path_images/ f) for f in os.listdir(path_images)
]
annot_paths = [
    str(path_annot/ f) for f in os.listdir(path_annot)
]


In [193]:
image_paths.sort()
annot_paths.sort()

In [194]:
image_size = 224
# loop over the annotations and images, preprocess them and store in lists
types = ['with_mask', 'without_mask', 'mask_weared_incorrect']

size_pattern = (16,16)
model_boxes = 4
images = []

AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32

https://towardsdatascience.com/yolo-v3-explained-ff5b850390f
https://keras.io/examples/vision/object_detection_using_vision_transformer/
https://keras.io/examples/vision/xray_classification_with_tpus/

In [200]:
bbox = []

def populateArray(i,information):
    classification = np.zeros(1+3+4)
    classification[0] = 1
    classification[1+types.index(i.find("name").text)] = 1
    classification[3+0] = int(i.find("bndbox").find("xmin").text)/information["size"][0]
    classification[3+1] = int(i.find("bndbox").find("ymin").text)/information["size"][1]
    classification[3+2] = int(i.find("bndbox").find("xmax").text)/information["size"][0]
    classification[3+3] = int(i.find("bndbox").find("ymax").text)/information["size"][1]
    return classification
    

    
def populateFullDetectionMatrix(targets,information):
    size = information["size"]
    classificationImage = np.zeros((size_pattern[0]*size_pattern[1]*model_boxes,7))
    for i in targets:
        x = (i[3]+i[5])*size_pattern[0]/2
        y = (i[4]+i[7])*size_pattern[1]/2
        row = classificationImage[(int(y)*size_pattern[0]+int(x))*model_boxes]
        offset = -1
        for index_free in range(0,7*model_boxes,7):   
            if(classificationImage[(int(y)*size_pattern[0]+int(x))*model_boxes+index_free][0]!=1):
                offset = index_free
        if(offset == -1):
            print(f"Error collision {len(targets)} on indexes {int(x)} and {int(y)}")
        for set_index in range(7): 
            classificationImage[(int(y)*size_pattern[0]+int(x))*model_boxes+offset][set_index] = i[set_index]
    return classificationImage
            

x = defaultdict(int)
for i in range(0, len(annot_paths)):
    # Access bounding box coordinates
    filename = path_annot / annot_paths[i]
    annot = fromstring(open(filename).read())
    information = {
        "size":(0,0),
        "annotations":[]
    }
    targets = []
    size = annot.find("size")
    information["size"] = int(size.find("width").text),int(size.find("height").text)
    
    for i in annot.findall("object"):
        classification = populateArray(i,information)
        targets.append(classification)
    targets = populateFullDetectionMatrix(targets,information)
    bbox.append(targets)
    
bbox = np.array(bbox)

In [201]:
print(bbox.shape)

(853, 1024, 7)


In [202]:
dataset_img_text = tf.data.Dataset.from_tensor_slices(image_paths)
dataset_img_bbox = tf.data.Dataset.from_tensor_slices(bbox)

In [203]:
def map_img(img_path):
    data = tf.io.read_file(img_path)
    img = tf.io.decode_png(data)
    img = tf.image.resize(img,(image_size,image_size))
    img = tf.cast(img,tf.dtypes.bfloat16)/255
    img.set_shape((224,224,3))
    return img

def map_bbox(bbox):
    bbox = tf.cast(bbox,tf.dtypes.bfloat16)
    return bbox

In [204]:
dataset_img_img = dataset_img_text.map(map_img)
dataset_img_bbox = dataset_img_bbox.map(map_bbox)

In [205]:
dataset = tf.data.Dataset.zip((dataset_img_img,dataset_img_bbox))

In [157]:


backbone = tf.keras.applications.efficientnet.EfficientNetB2(
        include_top=False, input_shape=[224, 224, 3]
    )


31801344/31790344 [==============================] - 2s 0us/step


In [ ]:
backbone.summary()

In [166]:
def get_backbone():
    """Builds ResNet50 with pre-trained imagenet weights"""
    backbone = tf.keras.applications.efficientnet.EfficientNetB2(
        include_top=False, input_shape=[224, 224, 3]
    )

    d4_output, d5_output, e6_output = [
        backbone.get_layer(layer_name).output
        for layer_name in ["block4d_add", "block5d_add", "block6e_add"]
    ]
    return tf.keras.Model(
        inputs=[backbone.inputs], outputs=[d4_output, d5_output, e6_output]
    )


In [311]:
class FeaturePyramid(tf.keras.layers.Layer):
    """Builds the Feature Pyramid with the feature maps from the backbone.

    Attributes:
      num_classes: Number of classes in the dataset.
      backbone: The backbone to build the feature pyramid from.
        Currently supports ResNet50 only.
    """

    def __init__(self, backbone=None, **kwargs):
        super().__init__(name="FeaturePyramid", **kwargs)
        self.backbone = backbone if backbone else get_backbone()
        self.conv_c3_1x1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv_c4_1x1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv_c5_1x1 = tf.keras.layers.Conv2D(256, 1, 1, "same")
        self.conv_c3_3x3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.conv_c4_3x3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.conv_c5_3x3 = tf.keras.layers.Conv2D(256, 3, 1, "same")
        self.conv_c6_3x3 = tf.keras.layers.Conv2D(256, 3, 2, "same")
        self.conv_c7_3x3 = tf.keras.layers.Conv2D(256, 3, 2, "same")
        self.upsample_2x = tf.keras.layers.UpSampling2D(2)

    def call(self, images, training=False):
        c3_output, c4_output, c5_output = self.backbone(images, training=training)
        p3_output = self.conv_c3_1x1(c3_output)
        p4_output = self.conv_c4_1x1(c4_output)
        p5_output = self.conv_c5_1x1(c5_output)
        p4_output = p4_output + self.upsample_2x(p5_output)
        p3_output = p3_output + p4_output
        p3_output = self.conv_c3_3x3(p3_output)
        p4_output = self.conv_c4_3x3(p4_output)
        p5_output = self.conv_c5_3x3(p5_output)
        return p3_output, p4_output, p5_output


In [302]:
def build_head(output_filters, bias_init):
    """Builds the class/box predictions head.

    Arguments:
      output_filters: Number of convolution filters in the final layer.
      bias_init: Bias Initializer for the final convolution layer.

    Returns:
      A keras sequential model representing either the classification
        or the box regression head depending on `output_filters`.
    """
    head = tf.keras.Sequential([tf.keras.Input(shape=[None, None, 256])])
    kernel_init = tf.initializers.RandomNormal(0.0, 0.01)
    for _ in range(4):
        head.add(
            tf.keras.layers.Conv2D(256, 3, padding="same", kernel_initializer=kernel_init)
        )
        head.add(tf.keras.layers.ReLU())
    head.add(
        tf.keras.layers.Conv2D(
            output_filters,
            3,
            1,
            padding="same",
            kernel_initializer=kernel_init,
            bias_initializer=bias_init,
        )
    )
    return head


In [303]:
class RetinaNet(tf.keras.Model):
    """A subclassed Keras model implementing the RetinaNet architecture.

    Attributes:
      num_classes: Number of classes in the dataset.
      backbone: The backbone to build the feature pyramid from.
        Currently supports ResNet50 only.
    """

    def __init__(self, num_classes, backbone=None, **kwargs):
        super().__init__(name="RetinaNet", **kwargs)
        self.fpn = FeaturePyramid(backbone)
        self.num_classes = num_classes

        prior_probability = tf.constant_initializer(-np.log((1 - 0.01) / 0.01))
        self.cls_head = build_head(num_classes, prior_probability)
        self.box_head = build_head(4, "zeros")

    def call(self, image, training=False):
        features = self.fpn(image, training=training)
        N = tf.shape(image)[0]
        cls_outputs = []
        box_outputs = []
        for feature in features:
            box_outputs.append(tf.reshape(self.box_head(feature), [N, -1, 4]))
            cls_outputs.append(
                tf.reshape(self.cls_head(feature), [N, -1, self.num_classes])
            )
        cls_outputs = tf.concat(cls_outputs, axis=1)
        box_outputs = tf.concat(box_outputs, axis=1)
        return tf.concat([box_outputs, cls_outputs], axis=-1)


In [312]:
model  = RetinaNet(3)

In [309]:
model.build(input_shape=(None,224,224,3))

In [294]:
model.summary()

Model: "RetinaNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FeaturePyramid (FeaturePyram multiple                  6394157   
_________________________________________________________________
sequential_42 (Sequential)   (None, None, None, 3)     1777155   
_________________________________________________________________
sequential_43 (Sequential)   (None, None, None, 4)     1779460   
Total params: 9,950,772
Trainable params: 9,900,861
Non-trainable params: 49,911
_________________________________________________________________


In [313]:
model.call(np.array([i for i in dataset_img_img.take(1)]))

Cleanup called...


<tf.Tensor: shape=(1, 147, 7), dtype=float32, numpy=
array([[[-2.23902520e-03,  7.37917516e-03,  1.20629035e-02, ...,
         -4.59080458e+00, -4.59137678e+00, -4.58650541e+00],
        [-9.10325442e-03,  7.54119176e-03,  6.10078406e-03, ...,
         -4.61826038e+00, -4.58748817e+00, -4.58296919e+00],
        [-1.10668838e-02, -4.33270913e-03, -4.82718972e-03, ...,
         -4.61144495e+00, -4.60202074e+00, -4.58423376e+00],
        ...,
        [ 4.54877689e-03, -2.42707366e-03,  6.84035243e-03, ...,
         -4.59595442e+00, -4.59896231e+00, -4.60137796e+00],
        [ 6.99355314e-03, -3.83618753e-03,  6.36196323e-03, ...,
         -4.59686470e+00, -4.59337091e+00, -4.60604429e+00],
        [ 4.84350603e-04,  1.42563437e-03,  1.30592864e-02, ...,
         -4.59278536e+00, -4.58993340e+00, -4.60114098e+00]]],
      dtype=float32)>